In [4]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import norm

In [ ]:
num = 1000  
event_time = int(num / 2) 

R_market = np.random.normal(0, 1, num) + np.arange(num) / num  
R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == int(num / 2) + 1) * 2  
results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit()  
alpha, beta = results.params  
resid = R_target - results.predict(sm.add_constant(R_market)) 
 
print(resid[event_time + 1] / resid[:event_time].std(ddof = 2)) 

0.8118941443978424


## Question 1

Which is closest to the probability that this t-test will be able to detect the event at event_time + 1 with the given code? 

In [8]:
# Parameters
num = 1000
event_time = int(num / 2)
threshold = 1.96  # Typical t-test threshold for 5% significance

# Simulate many times to estimate probability
trials = 10000
detections = []

for _ in range(trials):
    # Market return with drift
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num
    
    # Target return with +2 event at event_time + 1
    R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == event_time + 1) * 2

    # Regress pre-event data
    results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit()
    
    # Full residuals based on pre-event model
    resid = R_target - results.predict(sm.add_constant(R_market))
    
    # Standardized residual at event_time + 1
    t_stat = resid[event_time + 1] / resid[:event_time].std(ddof=2)
    
    # Record detection if t_stat exceeds threshold
    detections.append(t_stat > threshold)

# Estimate probability of detection
detection_probability = np.mean(detections)
print(f"Estimated probability of detection: {detection_probability:.3f}")


Estimated probability of detection: 0.514


## Question 2

Use the same code but put np.random.seed(0) at the beginning of each loop to ensure that you are performing placebo tests on a fixed dataset. Perform a placebo test by setting the fictitious event_time to all possible times, while leaving the event in R_target at just the 1 time. The placebo test trains itself on the data leading up to the fictitious event. About what fraction of placebo tests seem to detect an event at the fictitious event time? 

In [ ]:
# Constants
num = 1000
event_time = int(num / 2)
threshold = 1.96
placebo_results = []

# Generate a fixed dataset
np.random.seed(0)
R_market = np.random.normal(0, 1, num) + np.arange(num) / num
R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == event_time + 1) * 2

# Placebo test loop (avoiding edges)
for placebo_time in range(10, num - 2):
    # Train regression on data before placebo time
    results = sm.OLS(R_target[:placebo_time], sm.add_constant(R_market[:placebo_time])).fit()

    # Get residual at placebo_time + 1
    resid = R_target - results.predict(sm.add_constant(R_market))
    
    t_stat = resid[placebo_time + 1] / resid[:placebo_time].std(ddof=2)

    # Count false positive if above threshold
    placebo_results.append(t_stat > threshold)
    
# Final estimate
detection_probability = np.mean(placebo_results)
print(f"Estimated probability of detection: {detection_probability:.3f}")


Estimated probability of detection: 0.027


## Question 3

Do the same placebo test, but this time only run the test 20 times before and twenty times after the actual event. On average (over many runs of the code), what fraction of the 40 placebo tests get a higher t-value than the actual event? This time, adjust np.random.seed() to represent a different dataset when needed. 

In [16]:
# Constants
num = 1000
event_time = int(num / 2)  # 500
n_runs = 1000  # Number of simulations for averaging

exceed_fractions = []

for run in range(n_runs):
    np.random.seed(run)  # Different dataset each time

    # Generate data
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num
    R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == event_time + 1) * 2

    # Regress pre-event data
    results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit()
    # Full residuals based on pre-event model
    resid = R_target - results.predict(sm.add_constant(R_market))
    # Standardized residual at event_time + 1
    true_t_stat = resid[event_time + 1] / resid[:event_time].std(ddof=2)
   
    # 20 placebo times before and after the event
    placebo_times = list(range(event_time - 20, event_time)) + list(range(event_time + 2, event_time + 22))
    count_higher = 0

    for placebo_time in placebo_times:
       # Train regression on data before placebo time
        results = sm.OLS(R_target[:placebo_time], sm.add_constant(R_market[:placebo_time])).fit()
        # Get residual at placebo_time + 1
        resid = R_target - results.predict(sm.add_constant(R_market))        
        t_stat = resid[placebo_time + 1] / resid[:placebo_time].std(ddof=2)

        if t_stat > true_t_stat:
            count_higher += 1

    fraction = count_higher / len(placebo_times)   
    exceed_fractions.append(fraction)

# Average over all simulations
average_fraction = np.mean(exceed_fractions)
print(f"Average fraction of placebo t-stats exceeding the true event's: {average_fraction:.3f}")


Average fraction of placebo t-stats exceeding the true event's: 0.073


## Question 4

Do the same thing as in question 2, but this time use make_error with corr_const = 0.9 to generate the error for R_target instead of np.random.normal. Consider before attempting this: Would you expect this kind of dataset, where errors are not independent, to result in more or fewer false positives in the placebo tests? 

In [17]:
def make_error(corr_const, num):  
    sigma = 5 * 1 / np.sqrt((1 - corr_const)**2 / (1 - corr_const**2))  
    err = list() 
    prev = np.random.normal(0, sigma) 
    for n in range(num): 
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, sigma) 
        err.append(prev) 
    return np.array(err) 

In [18]:
# Constants
num = 1000
event_time = int(num / 2)
threshold = 1.96
placebo_results = []

# Generate a fixed dataset
np.random.seed(0)
R_market = np.random.normal(0, 1, num) + np.arange(num) / num
R_target = 2 + R_market + make_error(0.9, num) + (np.arange(num) == event_time + 1) * 2

# Placebo test loop (avoiding edges)
for placebo_time in range(10, num - 2):
    # Train regression on data before placebo time
    results = sm.OLS(R_target[:placebo_time], sm.add_constant(R_market[:placebo_time])).fit()

    # Get residual at placebo_time + 1
    resid = R_target - results.predict(sm.add_constant(R_market))
    
    t_stat = resid[placebo_time + 1] / resid[:placebo_time].std(ddof=2)

    # Count false positive if above threshold
    placebo_results.append(t_stat > threshold)
    
# Final estimate
detection_probability = np.mean(placebo_results)
print(f"Estimated probability of detection: {detection_probability:.3f}")

Estimated probability of detection: 0.011


## Homework Reflection 

1. Construct a dataset for an event study where the value, derivative, and second derivative of a trend all change discontinuously (suddenly) after an event.
Build a model that tries to decide whether the event is real (has a nonzero effect) using:
(a) only the value,
(b) the value, derivative, and second derivative.
Which of these models is better at detecting and/or quantifying the impact of the event?  (What might "better" mean here?)

2. Construct a dataset in which there are three groups whose values each increase discontinuously (suddenly) by the same amount at a shared event; they change in parallel
over time, but they have different starting values.  Create a model that combines group fixed effects with an event study, as suggested in the online reading.
Explain what you did, how the model works, and how it accounts for both baseline differences and the common event effect.